In [1]:
# Standard imports
import matplotlib.pyplot as plt
import numpy as np 

# abTEM imports to create the potentials.
from abtem.potentials import Potential
from abtem.dft import GPAWPotential

# Interactive widgets.
import ipywidgets as widgets

# Custom abTEM code to enable interactivity.
from abtem.visualize.interactive.artists import ImageArtist
from abtem.visualize.interactive.artists import LinesArtist
from abtem.visualize.interactive.utils import throttle
from abtem.visualize.interactive.canvas import Canvas
from abtem.visualize.utils import domain_coloring

In [20]:
# Canvases are layout areas where things are plotted.
canvas1 = Canvas(height=200, width=400, fig_margin={'top':10, 'bottom':50, 'left':60, 'right':20})
canvas2 = Canvas(height=200, width=400, fig_margin={'top':10, 'bottom':50, 'left':60, 'right':30})
canvas3 = Canvas(height=200, width=400, fig_margin={'top':10, 'bottom':50, 'left':60, 'right':20})
canvas4 = Canvas(height=200, width=400, fig_margin={'top':10, 'bottom':50, 'left':60, 'right':30})

# Artists do the plotting.
artist1 = ImageArtist(autoadjust_colorscale = False)
artist1._color_scale.min = 0
artist1._color_scale.max = 50
artist1.color_scheme = 'plasma'
canvas1.artists = {'image': artist1}

artist2 = ImageArtist(autoadjust_colorscale = False)
artist2._color_scale.min = -5
artist2._color_scale.max = 5
artist2.color_scheme = 'plasma'
canvas3.artists = {'image': artist2}

linesartist1 = LinesArtist()
linesartist2 = LinesArtist(colors='Blue')
linesartist3 = LinesArtist(colors='Black')

linesartist1._mark.labels = ['DFT']
linesartist2._mark.labels = ['IAM']
linesartist1._mark.display_legend = True
linesartist2._mark.display_legend = True
canvas2.figure.legend_location = 'top-right'

canvas2.artists = {'line1' : linesartist1, 'line2' : linesartist2}
canvas4.artists = {'line3' : linesartist3}

In [3]:
# Pre-calcutes the DFT step for different H-H distances. Takes about 

from ase import Atoms
from gpaw import GPAW, PW, FermiDirac
from gpaw import restart

a = 8.  # Size of unit cell (Angstrom)
c = a / 2 # Center

# Hydrogen molecule equilibrium bond length
d = 0.75  

# Gridpoints for the potential.
gpts = 128

# Initialize empty lists to store the potentials.
iampotentials = []
dftpotentials = []

# List of ten H-H separations to calculate.
distances = np.arange(0.75,3.25,0.25)

for i in np.arange(10):
    
    d = distances[i]
    
    # Set up an two H atoms at a distance d.
    atoms = Atoms('H2',
                  positions=([c - d / 2, c/2, c],
                             [c + d / 2, c/2, c]),
                  cell=(a, a/2, a))

    # Defines a GPAW planewave calculator.
    # A large cutoff is needed for plotting the potential near the nuclei accurately.
    calc = GPAW(mode=PW(800),
                basis='dzp',
                xc='PBE',
                occupations=FermiDirac(0.0, fixmagmom=True),
                txt=None
                )
    
    atoms.calc = calc
    atoms.get_potential_energy()
    
    # Prebuild and save the IAM and DFT potentials.
    iampotentials.append(Potential(atoms, gpts=gpts).build())
    dftpotentials.append(GPAWPotential(calc, gpts=gpts).build())

In [4]:
# Defines an interactive slider for the H-H distances.
slider = widgets.FloatSlider(description='H-H distance', min=0.75, max=3.0, value=0.75, step=0.25)

def slider_change(*args):
 
    # Pick the correct index of lists to correspond to the chosen distance.
    index = np.where(distances == slider.value)[0][0]
    
    # Project the potentials for plotting.
    iamprojected = iampotentials[index].project()
    dftprojected = dftpotentials[index].project()

    artist1.image = dftprojected.array
    artist2.image = (dftprojected.array - iamprojected.array) # Difference between DFT and IAM

    # Define a line profile through the center of the atoms along the x axis.
    shape = np.shape(iampotentials[index].array)
    iamline = iampotentials[index].array[shape[0]//2, :, shape[1]//2]
    dftline = dftpotentials[index].array[shape[0]//2, :, shape[1]//2]
    
    # Define line artists to draw the plots.
    linesartist1.x = iamprojected.calibrations[0].coordinates(gpts)
    linesartist1.y = iamline
    
    linesartist2.x = dftprojected.calibrations[0].coordinates(gpts)
    linesartist2.y = dftline

    linesartist3.x = linesartist2.x
    linesartist3.y = (dftline - iamline) # Difference between DFT and IAM

# Activate an observer to receive the interaction.
slider.observe(slider_change, 'value')

In [21]:
# Set the extents of the plots to match the cell extent
artist1.extent = ((0, a), (0, a/2))
artist2.extent = ((0, a), (0, a/2))

# Define y axis ranges for the plots.
canvas2.y_scale.min = -0.5
canvas2.y_scale.max = 50
canvas4.y_scale.min = -6
canvas4.y_scale.max = 1

# Define labels
canvas1.x_label = 'x [Å]'
canvas1.y_label = 'y [Å]'
canvas1.title = 'Projected potential' #units: (V*Å)

canvas3.x_label = 'x [Å]'
canvas3.y_label = 'y [Å]'
canvas3.title = 'Difference DFT – IAM' #units: (V*Å)

canvas2.x_label = 'x [Å]'
canvas2.y_label = 'Electrostatic potential (V)'
canvas2.title = 'Line profile'

canvas4.x_label = 'x [Å]'
canvas4.y_label = 'Electrostatic potential (V)'
canvas4.title = 'Difference DFT – IAM'

color_bar1 = artist1.get_color_bar(direction='vertical', margin=(50, 40))
color_bar2 = artist2.get_color_bar(direction='vertical', margin=(50, 40))

# Initializes the plot; otherwise user needs to touch the slider first.
slider_change()

# Set up and display the widgets in a grid.
widgets.VBox([widgets.HBox([canvas1.widget, color_bar1, canvas2.widget]),
              widgets.HBox([canvas3.widget, color_bar2, canvas4.widget]),
              slider])